In [1]:
#Support Vector Regression


In [2]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

### Data Import 

In [3]:
#Data Import
zipCSVPath = '../../data/Zip_Code_Data.csv'
zipDF = pd.read_csv(zipCSVPath)
zipDF['Zip'] = zipDF['Zip'].astype(str)
laDF = zipDF[(zipDF.City=="Los Angeles")]
laDFTarget = laDF[['Zip',
                    'Total_Pop',
                   'Percent_21_and_Older',
                   'Percent_60_and_Older',
                   'Percent_Male',
                   'Pct_Owner_Occupied',
                   'Pct_Renter_Occupied'
                  ]]

la_data = pd.read_csv('../../data/la_combined.csv').drop(columns='Unnamed: 0')
la_data['Zip'] = la_data['Zip'].astype(str)


la_data = la_data.merge(laDFTarget)

In [4]:
la_data_model = la_data.iloc[:103,:]
la_data_test = la_data.iloc[-20:,:]

In [32]:
la_data_model.head()

,Zip,Cluster_Labels,LAT,LNG,Num_Disp,Total_Households,Household_Median_Income,Total_Pop,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied
0,90001,2,33.973274,-118.248970,7,13669,43360,59832,63.3,10.9,51.7,35.9,64.1
1,90002,2,33.948951,-118.246980,1,12917,37285,53302,62.8,10.4,47.7,35.0,65.0
2,90003,2,33.963350,-118.273936,1,17484,40598,73730,63.0,10.0,49.0,28.3,71.7
3,90004,2,34.075721,-118.303017,1,22004,49675,60541,77.5,15.8,49.3,16.6,83.4
4,90005,3,34.059071,-118.303017,0,16781,38491,39732,78.6,16.8,50.4,7.7,92.3


In [31]:
la_data_test.head()

,Zip,Cluster_Labels,LAT,LNG,Num_Disp,Total_Households,Household_Median_Income,Total_Pop,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
103,91352,1,34.225510,-118.376517,26,12436,57145,47076,72.9,17.9,49.3,53.4,46.6,5.087365,0.804332
104,91356,2,34.169903,-118.540489,3,11802,76929,29822,78.7,27.1,48.6,56.8,43.2,3.623057,0.207686
105,91364,2,34.162012,-118.598469,4,10648,106225,27971,77.6,25.7,49.4,66.8,33.2,3.415476,0.146131
106,91367,2,34.178584,-118.609579,3,18433,96085,45970,78.9,22.7,47.3,53.1,46.9,5.242073,0.747358
107,91401,1,34.181152,-118.434827,3,14933,53882,39755,76.4,18.7,49.0,33.8,66.2,4.245392,0.415131


### Scale Data

In [7]:
#Map Domain data
X = la_data_model[['Total_Pop',
                   'Total_Households', 
                   'Household_Median_Income',
                   'Percent_21_and_Older',
                   'Percent_60_and_Older',
                   'Percent_Male',
                   'Pct_Owner_Occupied',
                   'Pct_Renter_Occupied']].to_numpy()
#Map Range data
y = la_data_model['Num_Disp'].to_numpy().reshape((-1,1))

In [8]:
#Scale Model Data
sc_X = StandardScaler()
sc_y = StandardScaler()

X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y).ravel()


In [9]:
#Scale Test Data
X_test = la_data_test[['Total_Pop',
                   'Total_Households', 
                   'Household_Median_Income',
                   'Percent_21_and_Older',
                   'Percent_60_and_Older',
                   'Percent_Male',
                   'Pct_Owner_Occupied',
                   'Pct_Renter_Occupied']].to_numpy()
y_test = la_data_test['Num_Disp'].to_numpy().reshape((-1,1))

X_test = sc_X.fit_transform(X_test)
y_test = sc_y.fit_transform(y_test).ravel()

### Grid Search to find optimal parameters for model

In [10]:
# Use grid Search to find the best parameters for the data

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'C':[0.5,1,5,10,50],'epsilon':[0.01,0.1,0.5,1,5,10,50]}

regr = svm.SVR()
clf = GridSearchCV(regr, parameters)
clf.fit(X,y)

clf.best_params_

{'C': 0.5, 'epsilon': 0.1, 'kernel': 'rbf'}

In [11]:
#Create a model with grid search parameters
regr = svm.SVR(**clf.best_params_)
regr.fit(X,y)

SVR(C=0.5)

### Testing the Model


In [64]:
def percentDiff(x1, y1):
    dummy = []
    for i,e in enumerate(x1):
        if e != 0:
            foo = abs(y1[i] - e) / e
        else:
            foo = y1[i]
        dummy.append(round(100*foo,2))
    return dummy

In [65]:
# Predict Values from model
y_pred = regr.predict(X_test)

y_pred = list(sc_y.inverse_transform(y_pred))
y_test = la_data_test['Num_Disp']


la_data_test['Predicted_Value'] = y_pred

la_data_test['Percent_Difference'] = percentDiff(y_test, y_pred)


c:\users\andrew\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\andrew\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [67]:
la_data_test[la_data_test['Percent_Difference'] < 40][['Num_Disp',
                                                       'Total_Pop',
                                                       'Total_Households', 
                                                       'Household_Median_Income',
                                                       'Percent_21_and_Older',
                                                       'Percent_60_and_Older',
                                                       'Percent_Male',
                                                       'Pct_Owner_Occupied',
                                                       'Pct_Renter_Occupied',
                                                       'Predicted_Value',
                                                       'Percent_Difference'
                                                      ]].describe()

,Num_Disp,Total_Pop,Total_Households,Household_Median_Income,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,5.666667,31998.666667,12440.500000,80690.833333,78.200000,20.233333,48.883333,42.683333,57.316667,4.594312,24.350000
std,2.422120,12336.908489,3398.772764,25443.843659,5.155967,5.320777,0.397073,17.000873,17.000873,1.294741,9.271377
min,3.000000,19672.000000,9493.000000,47037.000000,72.500000,14.500000,48.500000,25.100000,33.200000,3.415476,14.610000
25%,4.000000,26576.000000,10021.750000,63186.000000,74.075000,15.800000,48.525000,28.600000,45.600000,3.637111,16.885000
50%,5.000000,28896.500000,11225.000000,80468.500000,78.150000,19.650000,48.850000,40.150000,59.850000,4.151531,23.380000
75%,7.500000,32138.000000,13931.250000,100670.750000,80.800000,24.400000,49.175000,54.400000,71.400000,5.328227,29.342500
max,9.000000,55506.000000,18246.000000,111341.000000,86.000000,27.100000,49.400000,66.800000,74.900000,6.661236,38.680000


In [68]:
la_data_test[la_data_test['Percent_Difference'] > 40][['Num_Disp',
                                                       'Total_Pop',
                                                       'Total_Households', 
                                                       'Household_Median_Income',
                                                       'Percent_21_and_Older',
                                                       'Percent_60_and_Older',
                                                       'Percent_Male',
                                                       'Pct_Owner_Occupied',
                                                       'Pct_Renter_Occupied',
                                                       'Predicted_Value',
                                                       'Percent_Difference'
                                                      ]].describe()

,Num_Disp,Total_Pop,Total_Households,Household_Median_Income,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,6.285714,39705.785714,14314.428571,77122.142857,77.457143,19.128571,49.300000,42.257143,57.742857,4.912129,203.829286
std,8.835493,14566.543855,3829.256540,34076.289451,4.323943,3.596243,1.261867,14.575887,14.575887,0.733813,180.927420
min,0.000000,15292.000000,5600.000000,45796.000000,70.300000,14.700000,47.300000,17.900000,21.900000,3.691204,41.510000
25%,0.250000,30683.750000,12552.500000,53889.250000,74.525000,16.450000,48.450000,33.575000,48.425000,4.361159,64.810000
50%,2.500000,38298.000000,15009.000000,65581.500000,76.500000,18.950000,49.400000,38.800000,61.200000,4.994190,112.740000
75%,7.000000,46799.500000,16600.250000,89982.750000,80.700000,20.575000,50.050000,51.575000,66.425000,5.312092,356.810000
max,26.000000,72059.000000,19839.000000,174085.000000,84.900000,27.800000,51.700000,78.100000,82.100000,6.160512,512.770000


### Sacramento  data 


In [69]:
#Incorperate Other City Data to model

sacDF = zipDF[zipDF.City=="Sacramento"]
sacDFTarget = sacDF[['Zip', 
                   'LAT', 
                   'LNG', 
                   "Num_Disp",
                   'Total_Pop',
                   'Total_Households', 
                   'Household_Median_Income',
                   'Percent_21_and_Older',
                   'Percent_60_and_Older',
                   'Percent_Male',
                   'Pct_Owner_Occupied',
                   'Pct_Renter_Occupied'
                  ]]
sacDFTarget['Zip'] = sacDFTarget['Zip'].astype(str)

c:\users\andrew\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [70]:
#Scale City data
X2_test = sacDFTarget[['Total_Pop',
                   'Total_Households', 
                   'Household_Median_Income',
                   'Percent_21_and_Older',
                   'Percent_60_and_Older',
                   'Percent_Male',
                   'Pct_Owner_Occupied',
                   'Pct_Renter_Occupied']].to_numpy()

X2_test = sc_X.fit_transform(X2_test)



In [71]:
#Create prediction from previous model
y2_pred = regr.predict(X2_test)

In [72]:
y2_pred = list(sc_y.inverse_transform(y2_pred))
y2_test = sacDFTarget['Num_Disp']


sacDFTarget['Predicted_Value'] = y2_pred

sacDFTarget['Percent_Difference'] = percentDiff(y2_test, y2_pred)


c:\users\andrew\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\andrew\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [73]:
sacDFTarget[sacDFTarget['Percent_Difference'] <= 30][['Num_Disp',
                                                       'Total_Pop',
                                                       'Total_Households', 
                                                       'Household_Median_Income',
                                                       'Percent_21_and_Older',
                                                       'Percent_60_and_Older',
                                                       'Percent_Male',
                                                       'Pct_Owner_Occupied',
                                                       'Pct_Renter_Occupied',
                                                       'Predicted_Value',
                                                       'Percent_Difference'
                                                      ]].describe()

,Num_Disp,Total_Pop,Total_Households,Household_Median_Income,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#These zip codes least accuratly predict the number of dispensaries in Sac, so if our hypothesis that LA is the gold star 
#to which all cities should base their dispensary locations, then these would be ideal locations for new dispensaries. 
sacDFTarget[sacDFTarget['Percent_Difference'] > 30][['Num_Disp',
                                                       'Total_Pop',
                                                       'Total_Households', 
                                                       'Household_Median_Income',
                                                       'Percent_21_and_Older',
                                                       'Percent_60_and_Older',
                                                       'Percent_Male',
                                                       'Pct_Owner_Occupied',
                                                       'Pct_Renter_Occupied',
                                                       'Predicted_Value',
                                                       'Percent_Difference'
                                                      ]].describe()

,Num_Disp,Total_Pop,Total_Households,Household_Median_Income,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
count,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,1.450000,33087.350000,11854.700000,60847.350000,74.28500,18.055000,49.050000,45.150000,54.850000,5.254556,301.279000
std,1.431782,18059.943732,5308.868272,20053.753323,8.19077,4.291053,2.670403,15.379361,15.379361,1.405417,207.357054
min,0.000000,6294.000000,3147.000000,33938.000000,63.70000,13.300000,45.700000,7.200000,30.600000,2.277022,51.960000
25%,0.000000,19217.250000,8802.500000,48200.250000,68.72500,13.850000,47.050000,38.350000,45.775000,4.643110,135.095000
50%,1.000000,33366.500000,10834.500000,56522.500000,72.20000,18.100000,48.700000,47.500000,52.500000,5.551679,212.890000
75%,2.000000,40901.000000,15405.250000,66814.500000,77.92500,20.450000,49.925000,54.225000,61.650000,5.990743,468.472500
max,4.000000,79440.000000,23710.000000,106514.000000,92.00000,29.300000,57.900000,69.400000,92.800000,7.700432,770.040000


In [75]:
location = '../../data/sacDispDataSVR.csv'
sacDFTarget.to_csv(location)

In [76]:
sacDFTarget

,Zip,LAT,LNG,Num_Disp,Total_Pop,Total_Households,Household_Median_Income,Percent_21_and_Older,Percent_60_and_Older,Percent_Male,Pct_Owner_Occupied,Pct_Renter_Occupied,Predicted_Value,Percent_Difference
123,95814,38.58167,-121.49591,1.0,11908,6709,33938,92.0,21.5,57.9,7.2,92.8,5.593368,459.34
124,95815,38.60570,-121.44588,4.0,25673,9226,34583,69.2,13.7,51.5,33.7,66.3,6.391143,59.78
125,95824,38.51773,-121.44093,0.0,30296,9103,38985,67.1,15.7,49.8,39.5,60.5,5.173192,517.32
126,95832,38.44796,-121.49676,0.0,12114,3147,47341,65.5,15.8,50.7,49.3,50.7,4.958662,495.87
127,95823,38.47424,-121.44310,0.0,79440,23710,47553,67.3,15.7,48.6,45.7,54.3,7.700432,770.04
128,95838,38.64462,-121.44059,2.0,39053,11056,48416,66.5,13.3,50.3,50.3,49.7,5.722382,186.12
129,95817,38.55166,-121.45074,2.0,13758,6148,50925,81.6,18.0,45.7,40.7,59.3,3.427095,71.35
130,95820,38.53825,-121.44668,2.0,36437,12964,51068,73.7,18.7,48.9,53.0,47.0,5.509989,175.50
131,95828,38.48881,-121.40283,4.0,58717,17852,53229,70.9,19.8,48.8,61.9,38.1,7.688830,92.22
132,95616,38.56296,-121.81601,1.0,52212,17302,55510,63.7,13.6,46.3,34.9,65.1,4.826902,382.69
